In [ ]:
import pandas as pd
import numpy as np
import os
from ast import literal_eval
from gensim.models import Word2Vec
from preprocessing import dataset_add_columns
from raw_utils import save_to_csv

from sklearn.preprocessing import StandardScaler

In [26]:
# Path
cwd = os.getcwd()
csv_path = os.path.join(cwd, 'datasets/csv/')
train_token_file = 'train_dataset.csv'
test_token_file = 'test_dataset.csv'

In [27]:
train_tokens = pd.read_csv(os.path.join(csv_path, train_token_file), index_col=0, converters={'body': literal_eval})
test_tokens = pd.read_csv(os.path.join(csv_path, test_token_file), index_col=0, converters={'body': literal_eval})

In [28]:
train_tokens.head()

,label,body
0,0,"[possibly, friday, mon, laura, harder, daren, ..."
1,0,"[use, perl, daily, headline, mailer, week, per..."
2,0,"[andrew, lewis, portfolio, friday, june, view,..."
3,1,"[enter, track, number, track, pay, dear, custo..."
4,0,"[power, market, risk, reward, april, excalibur..."


In [29]:
def filter_vocab_words(wordlist, vocabulary):
    """
    Remove words not appearing in a vocabulary from a list.
    
    Parameters
    ----------
    wordlist : list of str
        The list of words to be filtered.
    vocabulary : list of str
        The vocabulary that will do the filtering.
        
    Returns
    -------
    list of str
        The filtered list.
    """
    return [word for word in wordlist if word in vocabulary]

def get_mean_vector(wordlist, word2vec_model):
    """
    Calculate the mean vector of a list of words.
    
    It takes the word vectors from the Word2Vec model and
    calculates the mean of those who appear in this specific
    list of words.
    
    Parameters
    ----------
    wordlist : list of str
        The list of words to be vectorized.
    word2vec_model : gensim.models.word2vec.Word2Vec
        The Word2Vec model that produced the word vectors.
        
    Returns
    -------
    numpy.ndarray
        An array containing the mean vector, or zeroes if
        the input wordlist was empty.
    """
    if len(wordlist) >= 1:
        return np.mean(word2vec_model.wv[wordlist], axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

In [30]:
def word2vec_features(text_col_train, text_col_test=None, vector_size=100, min_count=5, max_vocab_size=None, workers=1):
    """
    Extract Word2Vec embedding features using gensim.
    
    It uses the skip-gram model that Word2Vec provides. This is
    hardcoded (sg=1).
    
    Word2Vec represents each word in the corpus as a high-dimensional
    vector. Then, get_mean_vector() is used to get the averages of
    the vectors of all the words in an email, after removing the
    words that do not appear in the vocabulary that Word2Vec built.
    
    Some parameters for the vectorizer can also be passed as
    arguments.

    Parameters
    ----------
    token_text_col_train : pandas.Series of list of str
        The series that contains the tokenized emails as word lists.
    token_text_col_test : pandas.Series of list of str or None
        The series of the test set that contains the tokenized emails
        as word lists, or None if a test set is not provided.
    vector_size : int, default 100
        The size (dimensions) of the word vectors that will be
        produced. To be used by Word2Vec.
    min_count : int, default 5
        The minimum number of times a term has to appear in order to
        be included in the vocabulary. To be used by Word2Vec.
    workers : int, default 1
        How many threads to do the processing on. Note that if this is
        not set to 1, the processing witll be faster but the result
        will not be 100% reproducible. To be used by Word2Vec.
    max_vocab_size : int, default None
        The maximum number of terms in the vocabulary. To be used by
        Word2Vec.
    
    Returns
    -------
    dict
    {'vectorizer': gensim.models.word2vec.Word2Vec,
     'word2vec_train': pandas.DataFrame
     'word2vec_test': pandas.DataFrame or None}
        A dictionary that contains the vectorizer and the vectorized
        sets.
    
    See Also
    --------
    filter_vocab_words : Remove words not appearing in a vocabulary from a list.
    get_mean_vector : Calculate the mean vector of a list of words.
    """
    output = dict()
    
    # sg = 1 is the skip-gram model of Word2Vec default CBOW. Dự đoán ngữ cảnh của từ từ một từ hiện tại.
    # workers = 1 is the number of threads to do the processing on.
    # min_count = 5 is the minimum number of times a term has to appear in order to be included in the vocabulary.
    # vector_size = 100 is the size (dimensions) of the word vectors that will be produced.
    # max_final_vocab = None is the maximum number of terms in the vocabulary.
    model = Word2Vec(sentences=text_col_train,
                     min_count=min_count, vector_size=vector_size, max_final_vocab=max_vocab_size,
                     sg=1, workers=workers, seed=1746)
    
    # get the vocabulary of the model
    vocab = list(model.wv.key_to_index.keys())
    
    # filter the words that are not in the vocabulary
    filtered_col_train = text_col_train.apply(filter_vocab_words, vocabulary=vocab)
    # get the mean vector of the words in the email
    col_with_means_train = filtered_col_train.apply(get_mean_vector, word2vec_model=model)    
    word2vec_features_train = pd.DataFrame(col_with_means_train.tolist())
    
    output['vectorizer'] = model
    output['word2vec_train'] = word2vec_features_train
    
    if text_col_test is not None:
        filtered_col_test = text_col_test.apply(filter_vocab_words, vocabulary=vocab)
        col_with_means_test = filtered_col_test.apply(get_mean_vector, word2vec_model=model)    
        word2vec_features_test = pd.DataFrame(col_with_means_test.tolist())
        
        output['word2vec_test'] = word2vec_features_test
    else:
        output['word2vec_features_test'] = None

    return output

In [31]:
word2vec_dataset = word2vec_features(train_tokens['body'], test_tokens['body'], vector_size=100, min_count=5)

In [32]:
word2vec_train = word2vec_dataset['word2vec_train']
word2vec_test = word2vec_dataset['word2vec_test']
word2vec_model = word2vec_dataset['vectorizer']

In [33]:
word2vec_train.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.225819,0.141372,-0.223328,0.212550,-0.072821,-0.247794,0.280150,0.260777,-0.030664,0.038190,...,-0.228595,-0.199816,-0.154578,0.187148,0.195369,0.228151,-0.138362,-0.116963,0.011766,0.058782
1,-0.152479,0.001175,-0.015824,0.196747,0.124553,-0.396208,-0.021376,0.177244,-0.376067,-0.255902,...,-0.280072,-0.110481,-0.079562,0.305636,-0.057999,0.335967,-0.301189,0.066917,0.072775,-0.291711
2,-0.023560,-0.005283,-0.231558,0.155907,-0.148084,-0.036308,0.199756,0.029249,-0.028992,-0.131275,...,-0.424194,-0.018170,-0.267192,0.310382,-0.118429,0.335293,-0.173813,0.203804,0.121540,-0.041602
3,0.002166,0.113325,-0.275451,0.172201,0.078380,-0.201226,0.036213,-0.100815,0.115211,-0.219114,...,-0.619784,0.016422,-0.224874,0.302392,-0.295976,0.438338,-0.313398,0.205656,-0.105277,0.006415
4,-0.017440,-0.076307,-0.208194,0.044845,-0.032040,-0.093417,0.182971,-0.005259,0.005863,-0.159676,...,-0.420205,-0.084303,-0.103257,0.201104,-0.171942,0.370158,-0.212900,0.128863,0.058500,-0.038114


In [34]:
word2vec_test.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.083634,0.038720,-0.100804,0.105010,0.163783,-0.211749,-0.025200,0.045434,-0.224867,-0.011625,...,-0.337785,0.108491,-0.089312,0.168963,-0.493024,0.247047,-0.292622,-0.002336,-0.018909,0.083207
1,-0.116342,-0.005205,-0.094083,0.095255,0.086035,-0.249235,0.058704,0.042429,-0.251532,-0.068415,...,-0.350536,0.045148,-0.081383,0.207176,-0.279951,0.331935,-0.242790,0.046450,-0.042041,0.005624
2,-0.217088,0.070051,-0.038417,0.189789,-0.022470,-0.243347,0.248939,0.093826,-0.028668,0.074266,...,-0.242121,-0.075148,-0.014551,-0.023313,0.006544,0.247696,-0.212434,-0.006518,0.068123,0.134339
3,0.159065,-0.256510,-0.416176,0.360725,-0.077090,-0.601858,0.056939,0.397111,-0.013582,0.396469,...,-0.806288,-0.384959,-0.173511,0.055504,-0.345736,0.091994,-0.111311,-0.161888,-0.601564,-0.250697
4,-0.220807,0.020686,-0.020023,0.091729,-0.120923,-0.136814,0.368501,0.027600,0.025513,-0.162623,...,-0.370482,-0.231210,-0.033128,0.331855,0.118262,0.287261,-0.098794,-0.071500,-0.052851,-0.091635


In [35]:
word2vec_model.wv.most_similar('dollar')

[('eighteen', 0.7810409069061279),
 ('million', 0.7739255428314209),
 ('millon', 0.768381655216217),
 ('sixteen', 0.7666813135147095),
 ('musd', 0.7597635984420776),
 ('thousand', 0.759587287902832),
 ('eighty', 0.7587467432022095),
 ('seventeen', 0.7562917470932007),
 ('milllion', 0.7549381852149963),
 ('usdm', 0.7449865937232971)]

In [36]:
final_word2vec_train = dataset_add_columns(word2vec_train, [train_tokens['label']], ['label'])
final_word2vec_test = dataset_add_columns(word2vec_test, [test_tokens['label']], ['label'])

In [37]:
final_word2vec_train.head()

,label,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,0,-0.225819,0.141372,-0.223328,0.212550,-0.072821,-0.247794,0.280150,0.260777,-0.030664,...,-0.228595,-0.199816,-0.154578,0.187148,0.195369,0.228151,-0.138362,-0.116963,0.011766,0.058782
1,0,-0.152479,0.001175,-0.015824,0.196747,0.124553,-0.396208,-0.021376,0.177244,-0.376067,...,-0.280072,-0.110481,-0.079562,0.305636,-0.057999,0.335967,-0.301189,0.066917,0.072775,-0.291711
2,0,-0.023560,-0.005283,-0.231558,0.155907,-0.148084,-0.036308,0.199756,0.029249,-0.028992,...,-0.424194,-0.018170,-0.267192,0.310382,-0.118429,0.335293,-0.173813,0.203804,0.121540,-0.041602
3,1,0.002166,0.113325,-0.275451,0.172201,0.078380,-0.201226,0.036213,-0.100815,0.115211,...,-0.619784,0.016422,-0.224874,0.302392,-0.295976,0.438338,-0.313398,0.205656,-0.105277,0.006415
4,0,-0.017440,-0.076307,-0.208194,0.044845,-0.032040,-0.093417,0.182971,-0.005259,0.005863,...,-0.420205,-0.084303,-0.103257,0.201104,-0.171942,0.370158,-0.212900,0.128863,0.058500,-0.038114


In [39]:
final_word2vec_test[:10]

,label,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,0,-0.083634,0.038720,-0.100804,0.105010,0.163783,-0.211749,-0.025200,0.045434,-0.224867,...,-0.337785,0.108491,-0.089312,0.168963,-0.493024,0.247047,-0.292622,-0.002336,-0.018909,0.083207
1,0,-0.116342,-0.005205,-0.094083,0.095255,0.086035,-0.249235,0.058704,0.042429,-0.251532,...,-0.350536,0.045148,-0.081383,0.207176,-0.279951,0.331935,-0.242790,0.046450,-0.042041,0.005624
2,0,-0.217088,0.070051,-0.038417,0.189789,-0.022470,-0.243347,0.248939,0.093826,-0.028668,...,-0.242121,-0.075148,-0.014551,-0.023313,0.006544,0.247696,-0.212434,-0.006518,0.068123,0.134339
3,0,0.159065,-0.256510,-0.416176,0.360725,-0.077090,-0.601858,0.056939,0.397111,-0.013582,...,-0.806288,-0.384959,-0.173511,0.055504,-0.345736,0.091994,-0.111311,-0.161888,-0.601564,-0.250697
4,0,-0.220807,0.020686,-0.020023,0.091729,-0.120923,-0.136814,0.368501,0.027600,0.025513,...,-0.370482,-0.231210,-0.033128,0.331855,0.118262,0.287261,-0.098794,-0.071500,-0.052851,-0.091635
5,0,-0.114197,0.009651,-0.297519,0.167900,0.193371,-0.245883,0.064825,0.080239,-0.224032,...,-0.268436,0.026404,0.076453,0.167958,-0.359978,0.324179,-0.169458,0.205127,-0.092018,0.013378
6,0,-0.118323,0.016888,-0.165537,0.128755,-0.014106,-0.187088,0.156108,0.052935,-0.091449,...,-0.357732,-0.083979,-0.009297,0.105389,-0.210382,0.331292,-0.174561,0.164712,0.001465,0.005337
7,1,-0.026248,-0.076233,-0.157366,0.166218,-0.071650,0.004967,0.215168,0.074459,-0.090602,...,-0.513609,-0.045058,0.176644,0.270066,-0.203720,0.521550,-0.255628,0.250992,-0.067801,0.020297
8,1,0.151363,0.072165,-0.247575,0.060801,0.147160,-0.198635,0.105728,0.257638,-0.154305,...,-0.292172,-0.031772,0.154411,0.342699,0.044857,0.490920,-0.284738,-0.011826,-0.127554,-0.078167
9,1,0.007261,0.024153,-0.312892,0.101695,-0.047963,-0.226027,0.104286,0.059587,-0.023194,...,-0.497134,-0.124490,-0.159574,0.294030,-0.264181,0.388200,-0.190231,0.094726,-0.020672,-0.008831


In [42]:
save_to_csv(final_word2vec_train, csv_path, 'word2vec_train.csv')
save_to_csv(final_word2vec_test, csv_path, 'word2vec_test.csv')

Saving to /home/khanhnguyen/Public/workspace/python_3.10/project-phishing-email-detection/datasets/csv/word2vec_train.csv
Saving to /home/khanhnguyen/Public/workspace/python_3.10/project-phishing-email-detection/datasets/csv/word2vec_test.csv
